### Import library

In [1]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [2]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_1nPGD1QLYf8mFiVMZhXKjkUxzaGb



### Create list of alpha expressions
#### Step 1. Download datsets

In [33]:
#region in (USA), delay in (1), category in (other, news, socialmedia)

In [4]:
############################### Equity ,Region, Delay , Top ################################
datasets_df = hf.get_datasets(s,'EQUITY','USA','1') # by default we load all datasets USA TOP3000 delay 1
# get me the name of row equal Sentiment Data for Equity
# sentiment_data_name = datasets_df[datasets_df['name'] == 'Sentiment Data for Equity']
#sentiment_data_name

In [12]:
#get me all the rows that contain the word other, news, socialmedia in the category column and seperate each in a different dataframe
other_df = datasets_df[datasets_df['category'].str.contains('other', case=False)].reset_index(drop=True)
news_df = datasets_df[datasets_df['category'].str.contains('news', case=False)].reset_index(drop=True)
socialmedia_df = datasets_df[datasets_df['category'].str.contains('socialmedia', case=False)].reset_index(drop=True)

In [31]:
selected_datasets_df =other_df[(other_df['alphaCount']> 10)& (other_df['userCount']>10) & (other_df['fieldCount']<10)]

In [32]:
selected_datasets_df

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount,researchPapers
5,other15,ETF Constituents,Historical and Daily Holdings information of ETFs,other,other-etf-models,USA,1,TOP3000,1.0000,None,4.0,22,98,3,[]
6,other17,Flow into Equities Data,This dataset provides a measure of strength of...,other,other-etf-models,USA,1,TOP3000,0.8847,None,2.0,92,710,4,[]
8,other179,Other179,"The dataset provides the rankings on stocks, s...",other,other-employee-data,USA,1,TOP3000,0.9450,None,4.0,14,1190,9,[]
9,other193,Systematic Hedging for Investors to Evade Larg...,Machine learning based stock selection model. ...,other,other-aiml-data,USA,1,TOP3000,0.8845,None,2.0,80,884,3,[]
10,other238,Event based sentiment and behavioral factors m...,The model is constructed by combining multiple...,other,other-event-data,USA,1,TOP3000,0.7152,None,3.0,35,254,2,[]
11,other246,Regional Service Data,"The dataset contains weekly, monthly, quarterl...",other,other-analyst-models,USA,1,TOP3000,0.9047,None,5.0,18,146,7,[]
15,other351,International sentiment analysis NLP model,NLP driven predictive model dataset. GINA scor...,other,other-aiml-data,USA,1,TOP3000,0.7552,None,3.0,51,634,5,[]
16,other378,Accounting governance indicator models,Quadrophobia underrepresentation of accounting...,other,other-employee-data,USA,1,TOP3000,0.6812,None,3.0,27,258,2,[]
20,other407,Factors Dislocations Data,Risk factor impacted during COVID-19,other,other-event-data,USA,1,TOP3000,0.7613,None,3.0,34,326,8,[]
25,other427,Quantify text part in financial reports data,Quantify financial reports based on Natural La...,other,other-aiml-data,USA,1,TOP3000,0.7166,None,3.0,18,634,3,[]


In [20]:
# socialmedia_df

In [44]:
# # Filter rows where 'column_name' contains the substring 'kl'
# filtered_df = datasets_df[datasets_df['name'].str.contains('li', na=False)]
# filtered_df

In [8]:
# # select needed datasets
# selected_datasets_df = datasets_df[
#     (datasets_df["delay"] == 1) &
#     (datasets_df["coverage"] > 0.8) & (datasets_df["coverage"] <= 1) &
#     (datasets_df["fieldCount"] > 0) & (datasets_df["fieldCount"] < 1000) &
#     (datasets_df["region"] == 'USA') &
#     (datasets_df["universe"] == 'TOP3000') &
#     (datasets_df["userCount"] > 0) & (datasets_df["userCount"] < 100) &
#     (datasets_df["valueScore"] > 1) & (datasets_df["valueScore"] < 10) &
#     datasets_df["name"].str.contains('news', case=False) &
#     ((datasets_df["category"] == 'news') | (datasets_df["category"] == 'analyst'))
# ].sort_values(by=['valueScore'], ascending=False)
# selected_datasets_df

In [24]:
# Dán tên dataset vào đây thay cho 'analyst2'
selected_datasets_df = datasets_df[datasets_df['id']== 'socialmedia12']

In [25]:
selected_datasets_df

,id,name,description,category,subcategory,region,delay,universe,coverage,turnover,valueScore,userCount,alphaCount,fieldCount,researchPapers
225,socialmedia12,Sentiment Data for Equity,This dataset provides sentiment data with diff...,socialmedia,socialmedia-social-media,USA,1,TOP3000,0.9645,None,1.0,3219,10025,12,[{'title': 'Research Paper 01: Textual Sentime...


For demonstration we are picking the dataset with the highest value score, this is not mandatory but definitely a recommended practice

#### Step 2. Select the needed datafields


In [37]:
dataset_id = selected_datasets_df.id.values.tolist()# create a list of selected datasets ids, our list has only one element


In [38]:
all_data_field = []
for i in dataset_id:
    datafields_df = hf.get_datafields(s, dataset_id=i) # download all fields of dataset news
    all_data_field.append(datafields_df.id.values.tolist())

In [41]:
# merge all element in all_data_field into one list 
all_data_field = [item for sublist in all_data_field for item in sublist]

#### Step 3. Create expression list, using selected datafields
Here is an example: creating expressions that assign weights as per the time series skewness of vector average of the field value

In [43]:
# Thay Operators 
expression_list = [f'-vec_avg({x})' for x in all_data_field] # create a list of alpha expressions ts_skewness(vec_avg({x}),120) where x is a datafield id

#### Step 4. Apply generate_alpha function to the expression list
In generate alpha function you can specify region, universe, decay, delay and other simulation settings

In [44]:
alpha_list = []
for universe in ['TOP200','TOP3000','TOP500','TOP1000']:
    for decay_Value in range(1,10,1):
        for trun_value in [i/10 for i in range(0, 11)]:
            for neutralization_Value in ['MARKET','INDUSTRY','NONE','SUBINDUSTRY','SECTOR']:
                alpha_list.append([ace.generate_alpha(x, region= "USA", universe = universe, decay = decay_Value, neutralization = neutralization_Value, truncation= trun_value) for x in expression_list])

alpha_list = [item[0] for item in alpha_list]

In [45]:
len(alpha_list)

1980

In [23]:
# #when you send multiple alphas for simulation, please make sure all alphas of a single list should have common settings
# #alphas with different settings should be sent in a different list, for instance below list has all alphas with same settings

# alpha_list = [ace.generate_alpha(x, region= "USA", universe = "TOP3000",delay = 1, decay= 0, neutralization  = "INDUSTRY", truncation = 0.08, pasteurization = "ON",test_period = "P0Y0M0D",unit_handling = "VERIFY", nan_handling= "OFF") for x in expression_list]

# alpha_list[0]

In [47]:
import random

random_list = [random.randint(1, 1000) for _ in range(400)]

In [49]:
alpha_list = [alpha_list[i] for i in random_list]

This is an example - how alpha actually looks like when you send it to the platform.

### Simulate alpha list, get simulation result

simulate_alpha_list_multi will do a multi-simulation if list of alphas is greater than 10, which is the case here

the returned object will contain simulation results for all alphas as a list

In [51]:
#alpha expressions are sliced to first 10 for demonstration purpose
# Chạy nhiều hơn thì thay 10 thành số khác
result = ace.simulate_alpha_list_multi(s, alpha_list)

  7%|▋         | 9/134 [03:51<38:06, 18.29s/it]  

__Accessing the result of the first alpha, let's take a look at all the keys of this dictionary__

In [ ]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,...,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_competition,matches_themes
0,10366360,20000000,340,159,0.4488,0.1038,0.1519,0.000462,0.48,0.99,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
1,11183561,20000000,341,158,0.4520,0.1120,0.1822,0.000495,0.47,0.94,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
2,11494767,20000000,341,158,0.4545,0.1151,0.2040,0.000506,0.46,0.92,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
3,11494773,20000000,341,158,0.4545,0.1151,0.2040,0.000506,0.46,0.92,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
4,11494773,20000000,341,158,0.4545,0.1151,0.2040,0.000506,0.46,0.92,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,-27208232,20000000,0,500,0.1548,-0.2724,3.2154,-0.003520,-1.11,-0.84,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
986,-27208232,20000000,0,500,0.1548,-0.2724,3.2154,-0.003520,-1.11,-0.84,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
987,-27284092,20000000,0,500,0.1545,-0.2732,3.2230,-0.003536,-1.12,-0.84,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS
988,-27227642,20000000,0,500,0.1547,-0.2726,3.2174,-0.003524,-1.12,-0.84,...,-vec_avg(scl12_alltype_buzzvec),FAIL,PASS,FAIL,FAIL,FAIL,PASS,PASS,WARNING,PASS


In [63]:
result_st1.columns

Index(['pnl', 'book_size', 'long_count', 'short_count', 'turnover', 'returns',
       'drawdown', 'margin', 'fitness', 'sharpe', 'start_date', 'alpha_id',
       'expression', 'concentrated_weight', 'high_turnover',
       'is_ladder_sharpe', 'low_fitness', 'low_sharpe',
       'low_sub_universe_sharpe', 'low_turnover', 'matches_competition',
       'matches_themes'],
      dtype='object')

In [ ]:
# get 

In [62]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,date,endDate,limit,matched,name,result,startDate,themes,unmatched,value,year
0,Zokdb13,NaN,NaN,1.58,NaN,LOW_SHARPE,FAIL,NaN,NaN,NaN,0.740000,NaN
1,Zokdb13,NaN,NaN,1.00,NaN,LOW_FITNESS,FAIL,NaN,NaN,NaN,0.200000,NaN
2,Zokdb13,NaN,NaN,0.01,NaN,LOW_TURNOVER,PASS,NaN,NaN,NaN,1.207300,NaN
3,Zokdb13,NaN,NaN,0.70,NaN,HIGH_TURNOVER,FAIL,NaN,NaN,NaN,1.207300,NaN
4,Zokdb13,2019-07-10,NaN,0.10,NaN,CONCENTRATED_WEIGHT,FAIL,NaN,NaN,NaN,0.452952,NaN


In [56]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['Zokdb13' 'QQmNbAM' 'aOXQb85' 'WgKoeNd' '6EgoKzP' 'vmap2m3' 'WgKoe9d'
 'EKo8RKJ' 'jgNP8gk' 'oYQJopE' 'oYQJo5E' 'zWX2r18' 'kKeWJJd' '5vNAjj1'
 'oYQJoEE' 'aOXQV9x' 'jgNPekj' 'KLx5mok' 'dQe3a2v' 'mXe0aLK' 'p7QMaY6'
 'vmapAed' 'p7QManV' '2KPX5JN' 'mXe0aYE' 'dQe3aaX' 'weX2oNp' 'EKo8mrR'
 '8LgEGaq' '9RgdG9q' 'WgKoYZd' '5vNAGX6' 'xnX2lJw' 'EKo8NvJ' 'mXe0dmp'
 'WgKo3bx' 'A3qrevg' '2KPX9jP' 'O9KMRJg' 'YAXEEGl' 'Rr0llW1' 'EKo88AK'
 'aOXQQw1' 'aOXQQG1' '8LgEE27' '3AgkPrX' 'A3qrzMl' '7dgvm91' 'KLx5QNN'
 'P1jY8gM' 'LROmr59' 'b97rA9N' 'weX2WR1' '6EgoN9P' 'qXeYV3P' '2KPXeeZ'
 'WgKoJxk' 'EKo82Lm' 'p7QMp3v' '5vNA22M' 'l0J5N92' 'rWQNn8m' 'EKo82x9'
 'jgNPJEE' 'p7QMkrq' 'O9KMjPR' 'g37g5LM' 'vmapbod' 'aOXQ882' 'Rr0lwno'
 'GoEK5Jx' 'jgNPKQO' 'GoEK5g0' 'nWwe5vw' 'KLx5Mgj' 'ZokdVb8' 'Rr0lx6z'
 'O9KMdJg' 'QQmNROQ' 'vmap7pa' 'XKXvZz5' 'xnX2LVN' 'l0J5n52' 'g37glGJ'
 'p7QM2WX' 'YAXEVlR' 'ereVYVJ' '9Rgde8V' 'GoEK2Ko' '08nNVgv' '2KPXY5J'
 'JdRW93E' 'vmapQYb' 'aOXQpaO' 'P1jYVPJ' '9RgdnGd' 'EKo8dzJ' 'p